In [ ]:
! pip install transformers datasets accelerate evaluate optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 38.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
import transformers
import pandas as pd
from datasets import load_dataset
import evaluate # load_metric from datasets is removed
import numpy as np
import random
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import set_seed
import torch

In [ ]:
def set_all_seeds(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    set_seed(seed)  # HF helper (sets torch, random, numpy used inside HF)

In [ ]:
set_all_seeds(42)

In [ ]:
# loading a dataset from the csvs at https://github.com/inflaton/Explainable-Sentiment-Analysis-with-DeepSeek-R1/tree/main/dataset
data = load_dataset(
    "csv",
    data_files={
        "train": "GoEmotions-train.csv",
        "validation": "GoEmotions-val.csv",
        "test": "GoEmotions-test.csv",
    }
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
def train_model(model_checkpoint="distilbert-base-uncased",
                dataset=data,
                n_hpo_trials=0,
                batch_size=16,
                learning_rate=2e-5,
                num_train_epochs=5):
    """model_checkpoint: one of the huggingface model checkpoints
    dataset: A dataset object, e.g. from load_dataset
    n_hpo_trials: number of trials in the hyperparameter optimization
    batch_size: batch size before hyperparameter tuning
    learning_rate: learning rate before hyperparameter tuning
    num_train_epochs: number of epochs used in training before hyperparameter tuning

    Returns: None, pushes model to hub

    This function trains the model from start to finish, with hyperparameter tuning included if n_hpo_trials > 0.

    """
    accuracy = evaluate.load("accuracy")
    f1 = evaluate.load("f1")
    precision = evaluate.load('precision')
    recall = evaluate.load('recall')

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

    sentence1_key = "Text"

    # 1. Label maps
    labels = sorted(list(set(dataset["train"]["Emotion"])))
    label2id = {label: i for i, label in enumerate(labels)}
    id2label = {i: label for label, i in label2id.items()}

    # 2. Encode string labels to ints
    def encode_labels(example):
        example["labels"] = label2id[example["Emotion"]]
        return example

    dataset = dataset.map(encode_labels)

    # 3. Tokenization
    def preprocess_function(examples):
        return tokenizer(examples[sentence1_key], truncation=True)

    encoded_dataset = dataset.map(preprocess_function, batched=True)

    num_labels = len(labels) # 27 labels

    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                            num_labels=num_labels,
                                                            id2label=id2label,
                                                            label2id=label2id)
    metric_name = "f1_weighted"
    model_name = model_checkpoint.split("/")[-1]

    args = TrainingArguments(
        f"{model_name}-finetuned-GoEmotions",
        eval_strategy = "epoch",
        save_strategy = "epoch", # determines how often a model can be pushed
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model=metric_name,
        push_to_hub=True, # pushing to hugging face hub automatically is very helpful here
        hub_private_repo=True,
        logging_steps = 100,
        fp16 = True,
        report_to = 'none',
        disable_tqdm = False
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        acc = accuracy.compute(predictions=predictions, references=labels)

        f1_macro = f1.compute(
            predictions=predictions,
            references=labels,
            average="macro"
        )

        f1_weighted = f1.compute(
            predictions=predictions,
            references=labels,
            average="weighted"
        )

        recall_macro = recall.compute(
            predictions=predictions,
            references=labels,
            average="macro"
        )
        precision_macro = precision.compute(
            predictions=predictions,
            references=labels,
            average="macro"
        )

        return {
            "accuracy": acc["accuracy"],
            "f1_macro": f1_macro["f1"],
            "f1_weighted": f1_weighted["f1"],
            "recall_macro": recall_macro["recall"],
            "precision_macro": precision_macro["precision"]
        }

    trainer = Trainer(
        model,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )


    if n_hpo_trials == 0:
        trainer.train()
        trainer.push_to_hub() # making SURE it pushes to hub
        return None




    else:
        def model_init():
            return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

        args_hpo = TrainingArguments(
            f"{model_name}-finetuned-GoEmotions-hpo",  # separate dir
            eval_strategy = "epoch",
            save_strategy = "epoch",
            learning_rate=learning_rate,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=num_train_epochs,
            weight_decay=0.01,
            load_best_model_at_end=True,
            metric_for_best_model=metric_name,
            push_to_hub=False,
            fp16 = True,
            report_to = 'none',
            logging_strategy = 'no',
            disable_tqdm = True # disabling progress bar - seems to be an occasional rare bug in the progress bar where it divides by None
        )

        trainer = Trainer(
            model_init=model_init,
            args=args_hpo,
            train_dataset=encoded_dataset["train"],
            eval_dataset=encoded_dataset["validation"],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        def hp_space(trial):
            return {
                # BERT LR range, log scale
                "learning_rate": trial.suggest_float(
                    "learning_rate",
                    1e-5,
                    5e-5,
                    log=True
                ),

                "num_train_epochs": trial.suggest_int(
                    "num_train_epochs",
                    2,
                    8
                ),

                # Batch sizes
                "per_device_train_batch_size": trial.suggest_categorical(
                    "per_device_train_batch_size",
                    [8, 16, 32]
                ),
                #  weight decay range for AdamW
                "weight_decay": trial.suggest_float(
                    "weight_decay",
                    0.0,
                    0.3
                ),
                # Warmup fraction of total steps
                "warmup_ratio": trial.suggest_float(
                    "warmup_ratio",
                    0.0,
                    0.2
                )
            }

        def compute_objective(metrics):
            return metrics["eval_f1_weighted"]


        best_run = trainer.hyperparameter_search(n_trials=n_hpo_trials,
                                         direction="maximize",
                                         hp_space=hp_space,
                                         compute_objective=compute_objective)

        # rebuild trainer with push_to_hub=True, apply best hyperparams, and train once
        trainer = Trainer(
            model,
            args=args,
            train_dataset=encoded_dataset["train"],
            eval_dataset=encoded_dataset["validation"],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        # train after finding the best hyperparameters
        for n, v in best_run.hyperparameters.items():
            setattr(trainer.args, n, v)

        trainer.train()
        trainer.push_to_hub() # making SURE it pushes to hub
        return None











In [ ]:
%%time
train_model(model_checkpoint="distilbert-base-uncased", n_hpo_trials = 10)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...motions/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...motions/model.safetensors:  16%|#5        | 41.9MB /  268MB            

CPU times: user 39min 13s, sys: 38.4 s, total: 39min 52s
Wall time: 40min 5s


In [ ]:
%%time
train_model(model_checkpoint="bert-base-uncased", n_hpo_trials = 10)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/23485 [00:00<?, ? examples/s]

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...motions/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...motions/model.safetensors:  10%|9         | 41.8MB /  438MB            

CPU times: user 1h 41min 49s, sys: 1min 1s, total: 1h 42min 51s
Wall time: 1h 48min 48s


In [ ]:
%%time
train_model(model_checkpoint="bert-large-uncased", n_hpo_trials = 10)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/23485 [00:00<?, ? examples/s]

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...motions/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...motions/model.safetensors:   3%|3         | 41.9MB / 1.34GB            

CPU times: user 1h 32min 59s, sys: 1min 20s, total: 1h 34min 19s
Wall time: 2h 21min 38s


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("virtual12345/bert-large-uncased-finetuned-GoEmotions")
print("Model loaded successfully!")

config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Model loaded successfully!


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("virtual12345/bert-base-uncased-finetuned-GoEmotions")
print("Model loaded successfully!")

config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Model loaded successfully!


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("virtual12345/distilbert-base-uncased-finetuned-GoEmotions")
print("Model loaded successfully!")

config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Model loaded successfully!
